In [0]:
import pyspark
from pyspark.sql import SparkSession    
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
import numpy as np

In [0]:
spark = SparkSession.builder.appName('cruise_ship_linear_regression').getOrCreate()
data = spark.read.csv('/FileStore/tables/CourseDATA/cruise_ship_info.csv' , inferSchema=True , header = True)

In [0]:
data.show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 5 rows

In [0]:
data.columns

Out[4]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [0]:

indexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_Line_Index")
data_indexed = indexer.fit(data).transform(data)


data_indexed.select("Cruise_Line_Index").distinct().count()#20 different types of cruiselines have been indexed


Out[5]: 20

In [0]:
assembler = VectorAssembler(
 inputCols=[ "Cruise_Line_Index",
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density'] ,
  outputCol="Features"
                           )

output = assembler.transform(data_indexed)

In [0]:
output.columns

Out[7]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_Line_Index',
 'Features']

In [0]:
final_data = output.select(["features" , "crew"])


In [0]:
train_data , test_data = final_data.randomSplit([0.7,0.3],24)

In [0]:
lr = LinearRegression(labelCol='crew')

In [0]:
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.18,0.10,0.05]).addGrid(lr.fitIntercept, [False, True]).addGrid(lr.elasticNetParam, [0.5 , 0.3, 1, 0.7,0]).addGrid(lr.maxIter, [11,7,8]).addGrid(lr.standardization,[False,True]).addGrid(lr.solver,["auto", "normal", "l-bfgs"]).build()

tvs = TrainValidationSplit(estimator=lr,
                           collectSubModels=True,
                           estimatorParamMaps=paramGrid,
                           evaluator=RegressionEvaluator(labelCol='crew'),
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.8)

lr_model = tvs.fit(train_data)

/databricks/spark/python/pyspark/ml/util.py:762: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [0]:
best_cruise_model = lr_model.bestModel
best_cruise_model = best_cruise_model.evaluate(test_data)

In [0]:
best_cruise_model.r2

Out[13]: 0.9247220394819897

In [0]:
best_cruise_model.rootMeanSquaredError

Out[14]: 0.7899313089436834

In [0]:
best_cruise_model.meanSquaredError

Out[15]: 0.6239914728494809

In [0]:
lr_model.bestModel.extractParamMap()

Out[16]: {Param(parent='LinearRegression_37d1c1e7faf8', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2,
 Param(parent='LinearRegression_37d1c1e7faf8', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.7,
 Param(parent='LinearRegression_37d1c1e7faf8', name='epsilon', doc='The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber'): 1.35,
 Param(parent='LinearRegression_37d1c1e7faf8', name='featuresCol', doc='features column name.'): 'features',
 Param(parent='LinearRegression_37d1c1e7faf8', name='fitIntercept', doc='whether to fit an intercept term.'): True,
 Param(parent='LinearRegression_37d1c1e7faf8', name='labelCol', doc='label column name.'): 'crew',
 Param(parent='LinearRegression_37d1c1e7faf8', name='loss', doc='The loss function to be optimized. Supported options: squaredError, huber.'): 'squaredError',
 Param(parent='LinearRegression_37d1c1e7faf8', name='maxIter', doc='max number of iterations (>= 0).'): 7,
 Param(parent='LinearRegression_37d1c1e7faf8', name='predictionCol', doc='prediction column name.'): 'prediction',
 Param(parent='LinearRegression_37d1c1e7faf8', name='regParam', doc='regularization parameter (>= 0).'): 0.05,
 Param(parent='LinearRegression_37d1c1e7faf8', name='solver', doc='The solver algorithm for optimization. Supported options: auto, normal, l-bfgs.'): 'l-bfgs',
 Param(parent='LinearRegression_37d1c1e7faf8', name='standardization', doc='whether to standardize the training features before fitting the model.'): True,
 Param(parent='LinearRegression_37d1c1e7faf8', name='tol', doc='the convergence tolerance for iterative algorithms (>= 0).'): 1e-06}

In [0]:
default_model = lr.fit(train_data)

In [0]:
default_results = default_model.evaluate(test_data)

print(default_results.rootMeanSquaredError,
default_results.r2,
default_results.meanSquaredError,
default_results.meanAbsoluteError)

0.8098180487074236 0.9208840416542787 0.6558052720122991 0.6532167473270485